# One Million Documents: Benchmark

In [1]:
import svs
import pandas as pd
import os
import random
import string
import numpy as np

In [2]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
)

In [3]:
N = 1_000_000

# Step 1: Create Synthetic Data

In [4]:
async def fake_embeddings(list_of_strings):
    n = len(list_of_strings)
    m = 1536   # <-- same as OpenAI's `text-embedding-3-small` embedding dimensionality
    v = np.random.random(size=(n, m))
    mags = np.sqrt((v * v).sum(axis=1)).reshape(n, 1)
    return (v / mags).tolist()

In [5]:
%%time

letters = list(string.ascii_letters)

def rand_word():
    n = random.randint(1, 5)
    return ''.join(random.choice(letters) for _ in range(n))

def rand_document():
    n = random.randint(100, 500)
    return ' '.join(rand_word() for _ in range(n))

rand_docs = [rand_document() for _ in range(N)]

len(rand_docs), np.mean([len(doc) for doc in rand_docs])

CPU times: user 9min 42s, sys: 904 ms, total: 9min 43s
Wall time: 9min 44s


(1000000, np.float64(1199.319583))

# Step 2: Insert into Knowledge Base

In [6]:
DB_FILE_PATH = './onemillion.sqlite'

if os.path.exists(DB_FILE_PATH):
    os.unlink(DB_FILE_PATH)

kb = svs.KB(DB_FILE_PATH, fake_embeddings)

2024-07-17 03:31:22,669 - svs.kb - WARNING - Cannot store this non-standard embeddings function to the database. That's okay, but you'll have to explicitly pass this function to all future instantiations of this database.


In [7]:
with kb.bulk_add_docs() as add_doc:
    for doc in rand_docs:
        add_doc(doc)

2024-07-17 03:31:22,674 - svs.kb - INFO - starting bulk-add (as new database transaction)
2024-07-17 03:31:30,397 - svs.kb - INFO - getting 1000000 document embeddings...
2024-07-17 03:34:21,233 - svs.kb - INFO - *DONE*: got 1000000 document embeddings
2024-07-17 03:34:21,234 - svs.kb - INFO - invalidating cached vectors; they'll be re-built next time you `retrieve()`
2024-07-17 03:34:21,234 - svs.kb - INFO - ending bulk-add (committing the database transaction)


In [8]:
kb.close()

2024-07-17 03:34:22,047 - svs.kb - INFO - invalidating cached vectors; they'll be re-built next time you `retrieve()`


# Step 3: Retrieve Top-100 Documents

In [9]:
kb = svs.KB(DB_FILE_PATH, fake_embeddings)

len(kb)

2024-07-17 03:34:22,065 - svs.kb - WARNING - Cannot store this non-standard embeddings function to the database. That's okay, but you'll have to explicitly pass this function to all future instantiations of this database.


1000000

In [10]:
%%time

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!! THIS IS THE FIRST RETRIEVAL; IT WILL BE SLOW BECAUSE THE VECTORS HAVE TO BE LOADED FROM DISK INTO RAM !!!
# !!!                         (SUBSEQUENT RETRIEVALS WILL BE FASTER)                                        !!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

_ = kb.retrieve('pizza', n=100)

2024-07-17 03:34:22,105 - svs.kb - INFO - retrieving 100 documents with query string: pizza
2024-07-17 03:34:22,109 - svs.kb - INFO - re-building cached vectors...
2024-07-17 03:36:03,526 - svs.kb - INFO - re-building cached vectors... DONE!
2024-07-17 03:36:03,527 - svs.kb - INFO - got embedding for query!
2024-07-17 03:36:03,742 - svs.kb - INFO - computed 1000000 cosine similarities
2024-07-17 03:36:03,744 - svs.kb - INFO - retrieved top 100 documents


CPU times: user 1min 35s, sys: 6.02 s, total: 1min 41s
Wall time: 1min 41s


In [11]:
%%time

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !!! THIS SUBSEQUENT RETRIEVAL WILL BE FAST BECAUSE IT WILL USE THE CACHED VECTORS ALREADY IN RAM !!!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

_ = kb.retrieve('taco', n=100)

2024-07-17 03:36:03,756 - svs.kb - INFO - retrieving 100 documents with query string: taco
2024-07-17 03:36:03,758 - svs.kb - INFO - using cached vectors
2024-07-17 03:36:03,759 - svs.kb - INFO - got embedding for query!
2024-07-17 03:36:03,974 - svs.kb - INFO - computed 1000000 cosine similarities
2024-07-17 03:36:03,976 - svs.kb - INFO - retrieved top 100 documents


CPU times: user 781 ms, sys: 3.48 ms, total: 784 ms
Wall time: 220 ms


In [12]:
kb.close()

2024-07-17 03:36:03,985 - svs.kb - INFO - invalidating cached vectors; they'll be re-built next time you `retrieve()`
